# 演習11

## 条件

売り手は,$m$ 個の財を出品し,買い手は,財1個あたりの評価額 $v_i$ を持つ。各買い手は,m 個までの財を需要する。

買い手 $i$ の財1個あたりの入札額を $b_i$ としたとき,財の割当ては次の最適化問題の最適解として導出される。

最適解を導出する際はDPを使用する。

$$
\begin{aligned}
\text{最大化}\quad & \sum_{i \in N} b_i x_i \\
\text{条件}\quad &
\begin{cases}
\displaystyle \sum_{i \in N} x_i \leq m, \\
x \geq 0
\end{cases}
\end{aligned}
\tag{1}
$$

また、財の割り当ては下記で示されるVCGメカニズムに従う。

全員が参加した時のオークションの割り当てを $\vec{x}^{*}$ 、特定の買い手 $j$ を除いたオークションの割り当てを、$\vec{x}^{-j}$ とする。
このとき買い手 $j$ の支払額 $p_j$ は下記のように計算される。

$$
p_j = 
\sum_{i \in N \setminus \{j\}} b_i x_i^{-j}
-
\left(
\sum_{i \in N} b_i x_i^* - b_j x_j^*
\right)
$$





In [2]:
from typing import List

class DPBasedVCGMechanism:
    n_bidders: int
    n_items: int
    bids: List[List[int]]  # 2D valuation matrix [bidder][item]

    def __init__(self):
        pass

# 演習12

# 演習13

# 演習14

# 演習15

クラスHngarianとして実装する

```Python
class Hungarian:
    def __init__(self,X):

```

## 入力X


$$
\mathbf{X}
=
\begin{bmatrix}
x_{1,1} & x_{1,2} & \cdots & x_{1,N} \\
x_{2,1} & x_{2,2} & \cdots & x_{2,N} \\
\vdots  & \vdots  & \ddots & \vdots  \\
x_{N,1} & x_{N,2} & \cdots & x_{N,N}
\end{bmatrix},
\qquad 1 \le i \le N,\; 1 \le j \le N
$$


## step1

## step2

クラスHangarian内にてdef _step(self)として定義する。

## step3

クラスHangarian内にてdef _step3(self)として定義する。

## step4


In [19]:
import numpy as np
from typing import List


class Hungarian:
    N: int  # the dimension of the profit matrix
    tmp_tensor: np.ndarray

    def __init__(self, profit_matrix):
        self.profit_matrix = profit_matrix
        self.cost_matrix = self.create_cost_matrix(profit_matrix)
        self.step1()
        self.ans_pos = self.step_2_3()
        self.ans, self.ans_mat = self.ans_calculation(profit_matrix, self.ans_pos)

    def create_cost_matrix(self, profit_matrix):
        self.N = profit_matrix.shape[0]
        max_value = np.max(profit_matrix)
        return max_value - profit_matrix

    def min_zero_row(self, zero_mat, mark_zero):
        """
        The function can be splitted into two steps:
        #1 The function is used to find the row which containing the fewest 0.
        #2 Select the zero number on the row, and then marked the element corresponding row and column as False
        """

        # Find the row
        min_row = [99999, -1]

        for row_num in range(zero_mat.shape[0]):
            if np.sum(zero_mat[row_num] == True) > 0 and min_row[0] > np.sum(
                zero_mat[row_num] == True
            ):
                min_row = [np.sum(zero_mat[row_num] == True), row_num]

        # Marked the specific row and column as False
        zero_index = np.where(zero_mat[min_row[1]] == True)[0][0]
        mark_zero.append((min_row[1], zero_index))
        zero_mat[min_row[1], :] = False
        zero_mat[:, zero_index] = False

    def mark_matrix(self, mat):
        """
        Finding the returning possible solutions for LAP problem.
        """

        # Transform the matrix to boolean matrix(0 = True, others = False)
        cur_mat = mat
        zero_bool_mat = cur_mat == 0
        zero_bool_mat_copy = zero_bool_mat.copy()

        # Recording possible answer positions by marked_zero
        marked_zero = []
        while True in zero_bool_mat_copy:
            self.min_zero_row(zero_bool_mat_copy, marked_zero)

        # Recording the row and column positions seperately.
        marked_zero_row = []
        marked_zero_col = []
        for i in range(len(marked_zero)):
            marked_zero_row.append(marked_zero[i][0])
            marked_zero_col.append(marked_zero[i][1])

        # Step 2-2-1
        non_marked_row = list(set(range(cur_mat.shape[0])) - set(marked_zero_row))

        marked_cols = []
        check_switch = True
        while check_switch:
            check_switch = False
            for i in range(len(non_marked_row)):
                row_array = zero_bool_mat[non_marked_row[i], :]
                for j in range(row_array.shape[0]):
                    # Step 2-2-2
                    if row_array[j] == True and j not in marked_cols:
                        # Step 2-2-3
                        marked_cols.append(j)
                        check_switch = True

            for row_num, col_num in marked_zero:
                # Step 2-2-4
                if row_num not in non_marked_row and col_num in marked_cols:
                    # Step 2-2-5
                    non_marked_row.append(row_num)
                    check_switch = True
        # Step 2-2-6
        marked_rows = list(set(range(mat.shape[0])) - set(non_marked_row))

        return (marked_zero, marked_rows, marked_cols)

    def step1(self):
        self.tmp_tensor = self.cost_matrix.copy()
        for i in range(self.N):
            min_value = min(self.tmp_tensor[i])
            for j in range(self.N):
                self.tmp_tensor[i][j] -= min_value
        for j in range(self.N):
            col_values = [self.tmp_tensor[i][j] for i in range(self.N)]
            min_value = min(col_values)
            for i in range(self.N):
                self.tmp_tensor[i][j] -= min_value

    def step_2_3(self):
        tmp_tensor = self.tmp_tensor
        num_zero: int = 0
        while num_zero < self.N:
            pos, marked_rows, marked_cols = self.mark_matrix(np.array(tmp_tensor))
            num_zero = len(marked_rows) + len(marked_cols)
            if num_zero < self.N:
                tmp_tensor = self.adjust_matrix(tmp_tensor, marked_rows, marked_cols)
        return pos

    def adjust_matrix(self, mat, cover_rows, cover_cols):
        cur_mat = mat
        non_zero_element = []

        # Step 4-1
        for row in range(len(cur_mat)):
            if row not in cover_rows:
                for i in range(len(cur_mat[row])):
                    if i not in cover_cols:
                        non_zero_element.append(cur_mat[row][i])
        min_num = min(non_zero_element)

        # Step 4-2
        for row in range(len(cur_mat)):
            if row not in cover_rows:
                for i in range(len(cur_mat[row])):
                    if i not in cover_cols:
                        cur_mat[row, i] = cur_mat[row, i] - min_num
        # Step 4-3
        for row in range(len(cover_rows)):
            for col in range(len(cover_cols)):
                cur_mat[cover_rows[row], cover_cols[col]] = (
                    cur_mat[cover_rows[row], cover_cols[col]] + min_num
                )
        return cur_mat

    def ans_calculation(self, mat, pos):
        total = 0
        ans_mat = np.zeros((mat.shape[0], mat.shape[1]))
        for i in range(len(pos)):
            total += mat[pos[i][0], pos[i][1]]
            ans_mat[pos[i][0], pos[i][1]] = mat[pos[i][0], pos[i][1]]
        return total, ans_mat

    def get_answer(self):
        return self.ans, self.ans_mat


def main():

    profit_matrix = np.array([[7, 8, 5, 6], [6, 5, 9, 10], [4, 1, 10, 7], [3, 4, 6, 5]])
    solver = Hungarian(profit_matrix.copy())
    ans, ans_mat = solver.get_answer()
    print(f"Linear Assignment problem result: {ans:.0f}\n{ans_mat}")


if __name__ == "__main__":
    main()

Linear Assignment problem result: 31
[[ 7.  0.  0.  0.]
 [ 0.  0.  0. 10.]
 [ 0.  0. 10.  0.]
 [ 0.  4.  0.  0.]]


In [54]:
import numpy as np

class Hungarian:
    def __init__(self):
        self.optimal = []

    # step1
    def _step1(self, mat):
        output_mat = np.zeros_like(mat)
        for i, row in enumerate(mat):
            output_mat[i] = row - np.min(row)
        return output_mat

    # step2
    def _step2(self, mat):
        zero_coordinate = []
        for i, row in enumerate(mat):
            zero_coordinate.extend([(i, j) for j, v in enumerate(row) if v == 0])
        check_row = []
        check_column = []
        for elem in zero_coordinate:
            if not elem[0] in check_row and not elem[1] in check_column:
                check_row.append(elem[0])
                check_column.append(elem[1])
        if len(check_row) != mat.shape[0]:
            return False, zero_coordinate
        return True, zero_coordinate

    # step3
    def _step3(self, mat, zero_coordinate):
        zero_list = zero_coordinate
        zero_count = {}
        line = []
        while(len(zero_list) > 0):
            for elem in zero_list:
                r = "r_" + str(elem[0])
                c = "c_" + str(elem[1])
                if r in zero_count: zero_count[r] += 1
                else: zero_count[r] = 1
                if c in zero_count: zero_count[c] += 1
                else: zero_count[c] = 1
            max_zero = max(zero_count.items(), key=lambda x:x[1])[0]
            line.append(max_zero)
            rc = max_zero.split("_")[0]
            num = max_zero.split("_")[1]
            if rc == 'r': zero_list = [v for v in zero_list if str(v[0]) != num]
            else: zero_list = [v for v in zero_list if str(v[1]) != num]
            zero_count = {}
        return line

    # step4
    def _step4(self, mat, line):
        output_mat = np.zeros_like(mat)
        line_r = []
        line_c = []
        for l in line:
            rc = l.split("_")[0]
            num = int(l.split("_")[1])
            if rc == 'r': line_r.append(num)
            else: line_c.append(num)
        line_cut_mat = np.delete(np.delete(mat, line_r, 0), line_c,1)
        mini = np.min(line_cut_mat)
        cross_point = [(i,j) for i in line_r for j in line_c]
        non_line_point = [(i,j) for i in range(0,mat.shape[0]) for j in range(0,mat.shape[0]) if i not in line_r if j not in line_c]
        for co in cross_point:
            mat[co] += mini
        for co in non_line_point:
            mat[co] -= mini
        return mat

    def compute(self, mat):
        mat = self._step1(mat)
        mat = self._step1(mat.T).T
        while(True):
            flag, zero_coordinate = self._step2(mat)
            if flag: break
            line = self._step3(mat, zero_coordinate)
            mat = self._step4(mat, line)
        r = []
        c = []
        for v in zero_coordinate:
            if v[0] not in r and v[1] not in c:
                self.optimal.append(v)
                r.append(v[0])
                c.append(v[1])
        return self.optimal

# 行列を宣言
a = [7,8,5,6]
b = [6,5,9,10]
c = [4,1,10,7]
d = [3,4,6,5]
mat = np.array([a,b,c,d])

h = Hungarian()
h.compute(mat)
# output
# [(0, 0), (1, 3), (2, 2), (3, 1)]


ValueError: zero-size array to reduction operation minimum which has no identity

# 演習16

ハンガリー法を用いて $(AP)$、$(AP)^{-i}$ を計算する


In [4]:
class HungarianVCGAllocator:
    n_bidders: int
    n_items: int
    values: List[List[int]]  # values[i][j] in [10, 100]
    ans_allocation: List[int] = []
    ans_payment: List[int] = []
    def __init__(self, n_bidders: int, n_items: int, values: List[List[int]]):
        self.n_cols = len(values[0])
        self.n_rows = len(values)
        if self.n_cols != n_items or self.n_rows != n_bidders:
            raise ValueError("Dimension of values does not match n_bidders and n_items.")
        self.n_bidders = n_bidders
        self.n_items = n_items
        self.values = values
        self.Vmax = self.to_maximization_matrix()
    def to_maximization_matrix(self) -> List[List[int]]:
        max_value = max(max(row) for row in self.values)
        return [[max_value - self.values[i][j] for j in range(self.n_items)] for i in range(self.n_rows)]   

sample_n_bidders = 3
sample_n_items = 3
sample_tensor = [
    [10, 20, 30],
    [40, 50, 60],
    [70, 80, 90]
]
test_allocator = HungarianVCGAllocator(sample_n_bidders, sample_n_items, sample_tensor)
print(test_allocator.Vmax)

[[80, 70, 60], [50, 40, 30], [20, 10, 0]]
